# Production stage: use files 0-6 for model traning

In [1]:
!ls raw_data


data_gathered_at_0.csv	   data_gathered_at_3.csv  initial_training_data.csv
data_gathered_at_1.csv	   data_gathered_at_4.csv  test_data.csv
data_gathered_at_2.csv	   data_gathered_at_5.csv  validation_test_data.csv
data_gathered_at_2_bk.csv  data_gathered_at_6.csv


In [2]:
!mkdir accum_data

In [26]:
!cp raw_data/initial_training_data.csv accum_data/accumulated_data.csv


In [27]:
!ls accum_data/

accumulated_data.csv


In [28]:
import pandas as pd
import numpy as np
import os

from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical


raw_data_path = 'raw_data/'
accum_data_path = 'accum_data/'
accum_file_name = accum_data_path+'accumulated_data.csv'

def create_model():
    model = models.Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['Emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'Pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label


def data_to_tf_data(df):
    image_array, image_label = prepare_data(df)
    images = image_array.reshape((image_array.shape[0], 48, 48, 1))
    images = images.astype('float32')/255
    labels = to_categorical(image_label)
    return images, labels


In [29]:
def add_to_accum_data(nmb):
    accum_data = pd.read_csv(accum_file_name)
    next_data_file = raw_data_path+'data_gathered_at_' + str(nmb)+'.csv'
    add_data = pd.read_csv(next_data_file)
    accum_data.append(add_data)
    accum_data.to_csv(accum_file_name)


In [30]:
nmb = 0
add_to_accum_data(nmb)

In [31]:
train = pd.read_csv(accum_file_name)
train_images, train_labels = data_to_tf_data(train)

val = pd.read_csv(raw_data_path+'validation_test_data.csv')
val_images, val_labels = data_to_tf_data(val)


model = create_model()
class_weight = dict(zip(range(0, 7), (((train['Emotion'].value_counts()).sort_index())/len(train['Emotion'])).tolist()))
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = class_weight,
                    epochs=12,
                    batch_size=64)

df = pd.read_csv(raw_data_path+'test_data.csv')
test_images, test_labels = data_to_tf_data(df)
test_loss, test_acc = model.evaluate(test_images, test_labels)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/12
6/6 [==============================] - 4s 632ms/step - loss: 0.3372 - accuracy: 0.2430 - val_loss: 1.8610 - val_accuracy: 0.2449
Epoch 2/12
6/6 [==============================] - 3s 525ms/step - loss: 0.3116 - accuracy: 0.2835 - val_loss: 1.8795 - val_accuracy: 0.2449
Epoch 3/12
6/6 [==============================] - 3s 572ms/step - loss: 0.3044 - accuracy: 0.2835 - val_loss: 1.9381 - val_accuracy: 0.2449
Epoch 4/12
6/6 [==============================] - 3s 530ms/step - loss: 0.3079 - accuracy: 0.2835 - val_loss: 2.0681 - val_accuracy: 0.2449
Epoch 5/12
6/6 [==============================] - 3s 539ms/step - loss: 0.3021 - accuracy: 0.2835 - val_loss: 1.8856 - val_accuracy: 0.2449
Epoch 6/12
6/6 [==============================] - 3s 567ms/step - loss: 0.3068 - accuracy: 0.2648 - val_loss: 1.9238 - val_accuracy: 0.2508
Epoch 7/12
6/6 [==============================] - 3s 533ms/step - loss: 0.3046 - accuracy: 0.2960 - val_loss: 1.8884 - val_accuracy: 0.2449
Epoch 8/12
6/6 [====

In [32]:
from tensorflow import keras

model.save("initial_model")


INFO:tensorflow:Assets written to: initial_model/assets


In [41]:
!dvc add initial_model/


Adding...                                                                       
!
  0%|          |Saving files                          0/4 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |.DwLC3VcarwAa9qLWXncsFf.tmp           0/1 [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.jy4vXLkQkyT7MAaCqtteMF.tmp     0.00/164k [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.ni5xkLCKaDc4tBnqiaPpTk.tmp    0.00/3.83M [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.iHDTQ34QjeBTNqbrwrcBLS.tmp    0.00/2.72k [00:00<?,       ?it/s]
100% Add|██████████████████████████████████████████████|1/1 [00:00,  2.25file/s]

To track the changes with git, run:

	git add initial_model.dvc
